# Looking at which proteins are in the same bins across cancer types

1. Load in the groups' unique differential expression results.
2. For each group, see which proteins are consistently included across cancer types.